In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('../Data/Spotify_with_genre.csv')

In [3]:
df.head()

,artist_name,track_name,artist_pop,popularity,release_date,genre,track_uri,artist_uri
0,A Thousand Horses,Preachin' To The Choir,37.0,0.0,2017-06-02,Rap,6JKj9seJCshLVCfBLFp7dS,55RI2GNCfyXr0f14uIdhwd
1,Water Liars,You Work Days I Work Nights,29.0,10.0,2012,dnb,7L7u4PrNt5WMtVI9lHWlLm,6k3UpifDbb2ox25edM5j3P
2,Chambao,Sera,44.0,31.0,2007-11-06,Rap,12EyOqMPMyAKlKSynICAGN,2qhLqZ1pkiUl5HNrU2Nz0R
3,Jeezy,Big Shit Poppin,69.0,19.0,2007-08-07,Rap,3tCDjKseLQJhShfbeg4m7K,4yBK75WVCQXej1p04GWqxH
4,La Arrolladora Banda El Limón De Rene Camacho,No La Voy A Engañar,74.0,54.0,2012,Rap,0UICwSsSpUt98XkeCfupFn,5bSfBBCxY8QAk4Pifveisz


In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
label_encoder = LabelEncoder()

In [6]:
df['artist_name'] = label_encoder.fit_transform(df['artist_name'])
df['genre'] = label_encoder.fit_transform(df['genre'])

In [7]:
df['release_date']=pd.to_datetime(df['release_date'], errors='coerce')

In [8]:
df.dropna(subset=['release_date'], inplace=True)

In [9]:
df['year'] = df['release_date'].dt.year

In [10]:
df.drop(columns=['release_date'], inplace=True)

In [11]:
df.head()

,artist_name,track_name,artist_pop,popularity,genre,track_uri,artist_uri,year
0,34,Preachin' To The Choir,37.0,0.0,1,6JKj9seJCshLVCfBLFp7dS,55RI2GNCfyXr0f14uIdhwd,2017
1,2911,You Work Days I Work Nights,29.0,10.0,2,7L7u4PrNt5WMtVI9lHWlLm,6k3UpifDbb2ox25edM5j3P,2012
2,485,Sera,44.0,31.0,1,12EyOqMPMyAKlKSynICAGN,2qhLqZ1pkiUl5HNrU2Nz0R,2007
3,1268,Big Shit Poppin,69.0,19.0,1,3tCDjKseLQJhShfbeg4m7K,4yBK75WVCQXej1p04GWqxH,2007
4,1525,No La Voy A Engañar,74.0,54.0,1,0UICwSsSpUt98XkeCfupFn,5bSfBBCxY8QAk4Pifveisz,2012


In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler()

In [14]:
numerical_features = ['artist_pop', 'popularity']
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [15]:
df.head()

,artist_name,track_name,artist_pop,popularity,genre,track_uri,artist_uri,year
0,34,Preachin' To The Choir,0.37,0.000000,1,6JKj9seJCshLVCfBLFp7dS,55RI2GNCfyXr0f14uIdhwd,2017
1,2911,You Work Days I Work Nights,0.29,0.108696,2,7L7u4PrNt5WMtVI9lHWlLm,6k3UpifDbb2ox25edM5j3P,2012
2,485,Sera,0.44,0.336957,1,12EyOqMPMyAKlKSynICAGN,2qhLqZ1pkiUl5HNrU2Nz0R,2007
3,1268,Big Shit Poppin,0.69,0.206522,1,3tCDjKseLQJhShfbeg4m7K,4yBK75WVCQXej1p04GWqxH,2007
4,1525,No La Voy A Engañar,0.74,0.586957,1,0UICwSsSpUt98XkeCfupFn,5bSfBBCxY8QAk4Pifveisz,2012


In [16]:
from sklearn.cluster import DBSCAN, HDBSCAN, OPTICS
# from sklearn.preprocessing import StandardScaler

In [17]:
features_for_clustering = ['artist_pop', 'popularity', 'year', 'genre', 'artist_name']
weights= {'artist_pop': 1, 'popularity': 2, 'year': 3, 'genre': 4, 'artist_name': 1}

df[features_for_clustering].head()

,artist_pop,popularity,year,genre,artist_name
0,0.37,0.000000,2017,1,34
1,0.29,0.108696,2012,2,2911
2,0.44,0.336957,2007,1,485
3,0.69,0.206522,2007,1,1268
4,0.74,0.586957,2012,1,1525


In [18]:
df['artist_pop'] = df['artist_pop']*weights['artist_pop']
df['popularity'] = df['popularity']*weights['popularity']
df['year'] = df['year']*weights['year']

In [19]:
from scipy.spatial.distance import hamming

def weighted_distance(x, y):
    num_dist = np.sqrt(
        weights['artist_pop'] * (x[0] - y[0])**2 +
        weights['popularity'] * (x[1] - y[1])**2 +
        weights['year'] * (x[2] - y[2])**2
    )
    # year_dist = hamming(x[2], y[2])
    genre_dist = hamming(x[3], y[3])
    artist_dist = hamming(x[4], y[4])
    # genre_dist = np.count_nonzero(x[3], y[3])/len(x[3])
    # artist_dist = np.count_nonzero(x[4], y[4])/len(x[4])
    cat_dist = weights['genre']*genre_dist + weights['artist_name']*artist_dist
    
    # return num_dist  + weights['genre']*genre_dist + weights['artist_name']*artist_dist
    return num_dist + cat_dist


In [20]:
# Initialize DBSCAN
dbscan = DBSCAN(eps=0.3, min_samples=2, n_jobs=-1)  # Adjust eps and min_samples as per your data
hdbscan=HDBSCAN(min_cluster_size=3, metric=weighted_distance, n_jobs=-1)

In [21]:
# Fit DBSCAN clustering
clusters = dbscan.fit_predict(df[features_for_clustering])


In [22]:
hclusters=hdbscan.fit_predict(df[features_for_clustering])

In [23]:
# Add cluster labels to dataframe
df['cluster'] = clusters
df['hcluster'] = hclusters

In [24]:
import matplotlib.pyplot as plt

def plotClusters(col,algoName):
    for idx,i in enumerate(features_for_clustering):
        for j in features_for_clustering[idx+1:]:
            if i==j:
                continue
            plt.figure(figsize=(6, 4))
            plt.scatter(df[i], df[j], c=df[col], cmap='viridis', marker='o', alpha=0.8)
            plt.xlabel(i)
            plt.ylabel(j)
            plt.title(f'{algoName} Clustering') 
            plt.colorbar(label='Cluster')
            plt.grid(True) 
            plt.show() 


In [25]:
# plotClusters('cluster','DBSCAN')

In [26]:
# Calculate centroid values
centroid_values = df.groupby('cluster')[features_for_clustering].mean()

print("Centroid values:")
print(centroid_values)

Centroid values:
         artist_pop  popularity         year     genre  artist_name
cluster                                                            
-1         0.530699    0.544594  6025.203849  1.694227  1519.355904
 0         0.690000    0.434783  6021.000000  1.000000  1268.000000
 1         0.740000    0.891304  6045.000000  1.000000  2227.000000
 2         0.680000    1.000000  5976.000000  1.000000  2466.000000
 3         0.700000    0.793478  6045.000000  1.000000  1016.000000
...             ...         ...          ...       ...          ...
 349       0.700000    0.880435  6012.000000  1.000000  1016.000000
 350       0.820000    0.010870  6039.000000  1.000000  1204.000000
 351       0.700000    0.000000  6027.000000  0.000000   867.000000
 352       0.790000    0.706522  6051.000000  1.000000  2816.000000
 353       0.330000    0.641304  5976.000000  1.000000  1564.000000

[355 rows x 5 columns]


In [27]:
# plotClusters('hcluster','HDBSCAN')

In [28]:
centroid_values = df.groupby('hcluster')[features_for_clustering].mean()

print("Centroid values:")
print(centroid_values)

Centroid values:
          artist_pop  popularity    year     genre  artist_name
hcluster                                                       
-1          0.559392    0.702422  6031.2  1.422785  1543.739241
 0          0.470000    0.000000  5700.0  3.000000  2671.000000
 1          0.706667    0.224638  5898.0  5.000000  2045.666667
 2          0.540714    0.816770  5952.0  1.428571  2041.785714
 3          0.716667    1.115942  5949.0  0.000000  1332.333333
...              ...         ...     ...       ...          ...
 491        0.770000    0.755435  6048.0  1.000000  1589.000000
 492        0.690000    0.934783  6048.0  1.000000  1583.333333
 493        0.711111    0.917874  6048.0  1.000000  2084.444444
 494        0.540000    0.094203  6048.0  1.000000  1825.500000
 495        0.550000    0.000000  6048.0  1.000000   938.000000

[497 rows x 5 columns]


In [29]:
# Count the number of clusters formed
num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)  # Exclude noise points if present (-1 indicates noise)
print("Number of clusters formed:", num_clusters)

Number of clusters formed: 354


In [30]:
num_clusters = len(set(hclusters)) - (1 if -1 in hclusters else 0)  # Exclude noise points if present (-1 indicates noise)
print("Number of clusters formed:", num_clusters)

Number of clusters formed: 496


Hierarchical clustering:

In [31]:
from sklearn.cluster import AgglomerativeClustering

In [32]:
# Hierarchical clustering
hc = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
hierarhical_clusters = hc.fit_predict(df[features_for_clustering])

c:\Users\Yuval\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [33]:
#Add cluster labels to dataframe
df['hierarchical_cluster'] = hierarhical_clusters

In [34]:
# Visualize clusters
# plt.figure(figsize=(8, 6))
# plt.scatter(df['year'], df['popularity'], c=df['hierarchical_cluster'], cmap='viridis', marker='o', alpha=0.8)
# plt.xlabel('Year')
# plt.ylabel('Track Popularity')
# plt.title('Hierarchical Clustering')
# plt.colorbar(label='Cluster')
# plt.grid(True)
# plt.show()

# plotClusters('hierarchical_cluster','Hierarchical')

In [35]:
# Count the number of clusters formed
num_clusters_hc = len(set(hierarhical_clusters))
print("Number of clusters formed by hierarchical clustering:", num_clusters_hc)

Number of clusters formed by hierarchical clustering: 3


In [36]:
optics=OPTICS(min_samples=3)

In [37]:
optics_clusters=optics.fit_predict(df[features_for_clustering])

c:\Users\Yuval\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_optics.py:995: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [38]:
df['ocluster'] = optics_clusters


In [39]:
# plotClusters('ocluster','OPTICS')

In [40]:
# Count the number of clusters formed
num_clusters_optics = len(set(optics_clusters)) - (1 if -1 in optics_clusters else 0)  # Exclude noise points if present (-1 indicates noise)
print("Number of clusters formed by OPTICS:", num_clusters_optics)

Number of clusters formed by OPTICS: 693


In [41]:
# Calculate centroid values
centroid_values = df.groupby('ocluster')[features_for_clustering].mean()

print("Centroid values:")
print(centroid_values)

Centroid values:
          artist_pop  popularity         year     genre  artist_name
ocluster                                                            
-1          0.531016    0.534642  6026.006452  2.138710  1503.563441
 0          0.676667    0.949275  6048.000000  1.000000    36.333333
 1          0.800000    1.347826  6045.000000  1.000000    38.000000
 2          0.623333    0.949275  6049.000000  1.000000    27.666667
 3          0.770000    1.057971  6045.000000  0.666667    20.333333
...              ...         ...          ...       ...          ...
 688        0.637500    0.907609  5904.000000  1.000000  2347.500000
 689        0.567143    0.282609  5892.000000  1.000000   868.857143
 690        0.740000    0.829193  5913.857143  1.857143  1558.000000
 691        0.463333    1.152174  5895.000000  3.000000  1611.333333
 692        0.470000    0.000000  5700.000000  3.000000  2671.000000

[694 rows x 5 columns]


In [42]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score, homogeneity_score, completeness_score, v_measure_score

algoNames=['DBSCAN','HDBSCAN','Hierarchical','OPTICS']
cols=['Silhouette Score', 'Davies Bouldin Score', 'Calinski Harabasz Score' ] #, 'Adjusted Rand Score', 'Homogeneity Score', 'Completeness Score', 'V Measure Score'
 
results = pd.DataFrame(columns=cols, index=algoNames)

def evalClusteringAlgo(algoName,clusters,features=features_for_clustering):
    silhouette = silhouette_score(df[features], clusters)
    davies_bouldin = davies_bouldin_score(df[features], clusters)
    calinski_harabasz = calinski_harabasz_score(df[features], clusters)
    # adjusted_rand = adjusted_rand_score(df['genre'], clusters)
    # homogeneity = homogeneity_score(df['genre'], clusters)
    # completeness = completeness_score(df['genre'], clusters)
    # v_measure = v_measure_score(df['genre'], clusters)
    results.loc[algoName] = [silhouette, davies_bouldin, calinski_harabasz] #, adjusted_rand, homogeneity, completeness, v_measure]
    
    

In [43]:
evalClusteringAlgo('DBSCAN',clusters)
evalClusteringAlgo('HDBSCAN',hclusters)
evalClusteringAlgo('Hierarchical',hierarhical_clusters) 
evalClusteringAlgo('OPTICS',optics_clusters)

results.head()

,Silhouette Score,Davies Bouldin Score,Calinski Harabasz Score
DBSCAN,-0.66501,2.859046,2.344913
HDBSCAN,-0.880222,241.881021,1.556707
Hierarchical,0.56605,0.497496,17123.231928
OPTICS,-0.006493,2.573982,11.671417


In [44]:
from sklearn.metrics import silhouette_score

# Calculate silhouette score for DBSCAN
silhouette_score_dbscan = silhouette_score(df[features_for_clustering], clusters)

# Calculate silhouette score for hierarchical clustering
silhouette_score_hc = silhouette_score(df[features_for_clustering], df['hierarchical_cluster'])

silhouette_score_hdbscan = silhouette_score(df[features_for_clustering], hclusters)

silhouette_score_optics = silhouette_score(df[features_for_clustering], optics_clusters)

print("Silhouette score for DBSCAN:", silhouette_score_dbscan)
print("Silhouette score for hierarchical clustering:", silhouette_score_hc)
print("Silhouette score for HDBSCAN:", silhouette_score_hdbscan)
print("Silhouette score for OPTICS:", silhouette_score_optics)

Silhouette score for DBSCAN: -0.6650097292247373
Silhouette score for hierarchical clustering: 0.5660496753254635
Silhouette score for HDBSCAN: -0.8802222044910336
Silhouette score for OPTICS: -0.006492933416917398
